In [ ]:
import warnings
from transformers import pipeline

warnings.simplefilter("ignore")
MAX_LENGTH = 400  # モデルに入力することができる文字数の上限
translator = pipeline("translation", model="staka/fugumt-en-ja")

## whisperから作成した字幕ファイルを読み込む
whisperはデフォルトで `.srt`ファイルを書き出します。このファイルの英語の部分を日本語に翻訳します。
翻訳の正しさは保証されておらず、また本コードも動作や出力に関しては一切の責任を負えません。

- [openai/whisper](https://github.com/openai/whisper)
- 翻訳に使用するモデル：[staka/fugumt-en-ja ](https://staka.jp/wordpress/?p=413)
- [Hugging Face](https://huggingface.co/staka/fugumt-en-ja)


In [ ]:
file = open("FlexLNG_Q3_2022_Key_Takeaways.mp3.srt", "r")
lines = [l.replace("\n", "") for l in file.readlines()]
lines[:15]

## 前処理
テキストに前処理を加える場合は、ファイルに保存する前にテキストを書き換えます。

In [ ]:
def zundamonize(text: str) -> str:
    """

        語尾をそれらしくする

        Args:
            text (str): 日本語テキスト

        Returns:
            str: 前処理済みの日本語テキスト

    """
    text = text.replace("Flex", "フレックス")

    if text.endswith("ありがとうございました"):
        return text.replace("ありがとうございました", "ありがとうなのだ。")
    elif text.endswith("しました。"):
        return text.replace("しました。", "したのだ。")
    elif text.endswith("れました。"):
        return text.replace("れました。", "れたのだ。")
    elif text.endswith("れました。"):
        return text.replace("れました。", "れたのだ。")
    elif text.endswith("れます。"):
        return text.replace("れます。", "れるのだ。")
    elif text.endswith("できます。"):
        return text.replace("できます。", "できるのだ。")
    elif text.endswith("あります。"):
        return text.replace("あります。", "あるのだ。")
    elif text.endswith("ようこそ。"):
        return text.replace("ようこそ。", "ようこそなのだ。")
    elif text.endswith("ています。"):
        return text.replace("ています。", "ているのだ。")
    elif text.endswith("ている。"):
        return text.replace("ている。", "ているのだ。")
    elif text.endswith("できる。"):
        return text.replace("できる。", "できるのだ。")
    elif text.endswith("できま。"):
        return text.replace("できます。", "できるのだ。")
    elif text.endswith("できた。"):
        return text.replace("できた。", "できたのだ。")
    elif text.endswith("えました。"):
        return text.replace("えました。", "えたのだ。")
    elif text.endswith("しました。"):
        return text.replace("しました。", "したのだ。")
    elif text.endswith("ました。"):
        return text.replace("ました。", "たのだ。")
    elif text.endswith("った。"):
        return text.replace("った。", "ったのだ。")
    elif text.endswith("した。"):
        return text.replace("した。", "したのだ。")
    elif text.endswith("する。"):
        return text.replace("する。", "するのだ。")
    elif text.endswith("です。"):
        return text.replace("です。", "なのだ。")
    return text

## 翻訳したテキストを出力する
字幕用のフォーマット(`.srt`)とテキストファイルの両方を出力します。
途中、タイムスタンプをいろいろ書き換えているのは字幕のタイミングを調整するためです。また、ファイル一行目に改行を入れるとAdobe Premire Pro等で読み取りに失敗するので注意してください。

参考文献：[SubRip](https://en.wikipedia.org/wiki/SubRip)

In [ ]:
cnt = 0
result = []
text_only_result = []
temp_timestamp_start = "00:00:00,000"
temp_timestamp_end = "00:00:00,000"
temp_text = ""
centense_continue = False

for line in lines:
    if len(line) == 0:
        # 空白行はスキップ
        continue
    elif line[0].isdigit():
        # 会話以外の出力はスキップ
        if "-->" in line:
            if centense_continue:
                temp_timestamp_end = line.split(" --> ")[1]
            else:
                temp_timestamp_start, temp_timestamp_end = line.split(" --> ")
            continue
        elif len(line) < 5:
            continue

    # 会話が続いているかの判定
    if line.endswith("."):
        centense_continue = False
    else:
        centense_continue = True

    # 翻訳
    temp_text += line
    if not centense_continue:
        cnt += 1
        translation_text = translator(temp_text)[0]["translation_text"]
        translation_text = zundamonize(translation_text)
        temp_text = ""

        print("")
        print(cnt)
        print(f"{temp_timestamp_start} --> {temp_timestamp_end}")
        print(translation_text)

        if cnt > 1:
            result.append("")
        result.append(cnt)
        result.append(f"{temp_timestamp_start} --> {temp_timestamp_end}")
        result.append(translation_text)
        text_only_result.append(f"{translation_text}")
    else:
        continue

with open("日本語字幕.srt", "w", encoding="utf-8-sig") as f:
    for line in result:
        f.write(f"{line}\n")

with open("日本語テキスト.txt", "w", encoding="utf-8-sig") as f:
    for line in text_only_result:
        f.write(f"{line}\n")